# 17  基于MindSpore构造Dropout层

本实验主要介绍使用MindSpore深度学习框架中使用Dropout层，并对比Dropout层对深度学习模型性能的影响

# 1 实验目的
- 掌握如何使用Mindspore深度学习框架构造Dropout层；
- 了解Dropout层的相关原理以及应用；
- 了解dropout层对于深度学习模型性能的影响;

# 2 Dropout层背景及知识点介绍
## 2.1背景知识
2012年，Hinton在论文[《Improving neural networks by preventing co-adaptation of feature detectors》](https://arxiv.org/pdf/1207.0580.pdf)中提出Dropout。当一个复杂的前馈神经网络被训练在小数据集时，容易造成过拟合。为了防止过拟合，可以通过阻止特征检测器的共同作用来提高神经网络的性能。同年，Alex、Hinton在论文[《ImageNet Classification with Deep Convolutional Neural Networks》](https://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=1166C42C86BD3A3C6C75B53E2CD2E14F?doi=10.1.1.299.205&rep=rep1&type=pdf)中使用了Dropout算法，用于防止过拟合。该论文赢得了2012年图像识别大赛冠军。随后，关于Dropout的论文[《Dropout:A Simple Way to Prevent Neural Networks from Overfitting》](http://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf)、[《Improving Neural Networks with Dropout》](http://www.cs.toronto.edu/~nitish/msc_thesis.pdf)、[《Dropout as data augmentation》](https://arxiv.org/pdf/1506.08700.pdf)等不断提高Dropout层的应用。

## 2.2Dropout原理介绍

Dropout作为训练深度神经网络的trick供开发者选择。其原理是在每个训练批次中，通过忽略一半的特征检测器（让一半的隐层节点值为0），从而地减少过拟合现象。这种方式可以减少特征检测器（隐层节点）间的相互作用（检测器相互作用是指某些检测器依赖其他检测器才能发挥作用）。具体的说：在前向传播时，让某个神经元的激活值以一定的概率p停止工作，从而使模型泛化性更强，减少对某些局部的特征的依赖。

![avatar](fig/fig1.png)


# 3 实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7。


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

# 4. 数据处理


## 4.1 数据准备

MNIST数据集是机器学习领域中经典的数据集，由60,000个训练样本和10,000个测试样本组成，每张图片都是28x28像素大小的灰度图像，数字从0到9。

MNIST数据集主要用于图像识别领域的研究，特别是用于机器学习算法的测试和比较。由于它的简单性和易于使用，MNIST已经成为机器学习社区中的标准数据集之一。

以下是几个示例图像：

![jupyter](./fig/Fig001.png)

每个图像都被转换成784个数字的一维向量（28×28=784）。这些数字表示像素的灰度值，值的范围从0（黑色）到255（白色）。

从开放数据集中下载MNIST数据集的压缩包,并解压储存在项目的根目录下。

In [16]:
# 从开放数据集中下载MNIST数据集
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:01<00:00, 10.2MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


下载后的数据集目录结构如下：



/MNIST_Data\
├── train\
&emsp;├──train-images-idx3-ubyte\
&emsp;└──train-labels-idx1-ubyte\
├──text\
&emsp;├──t10k-images-idx3-ubyte\
&emsp;└──t10k-labels-idx1-ubyte

## 4.2 设置超参数

集中设置在模型训练过程中需要使用到的超参数的具体数值，方便在后续调试代码时修改超参数。

In [125]:
BATCH_SIZE= 64       # batch的大小
LEARNING_RATE = 1e-2 # 学习率
EPOCH = 200      # 迭代次数

## 4.3 获取数据集对象
利用mindspore中的dataset相关的函数读取数据集中的数据。

In [126]:
from mindspore.dataset import transforms
# 读取和解析Manifest数据文件构建数据集
from mindspore.dataset import MnistDataset
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

## 4.4 数据处理 
MindSpore的dataset使用数据处理流水线（Data Processing Pipeline），需指定map、batch、shuffle等操作。这里我们使用map对图像数据及标签进行变换处理，然后将处理好的数据集打包为大小为64的batch。

In [127]:
# MindSpore库
import mindspore
# 神经网络模块
from mindspore import nn
# 常见算子操作
from mindspore import ops
# 图像增强模块
from mindspore.dataset import vision
def datapipe(dataset, batch_size):
    image_transforms = [
        # 基于给定的缩放和平移因子调整图像的像素大小。输出图像的像素大小为：output = image * rescale + shift。
        # 此处rescale取1.0 / 255.0，shift取0
        vision.Rescale(1.0 / 255.0, 0),
        # 正则化 均值为0.1307，标准差为0.3081（查自官网）
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        # 将输入图像的shape从 <H, W, C> 转换为 <C, H, W>
        vision.HWC2CHW()
    ]
    # 将输入的Tensor转换为指定的数据类型。
    label_transform = transforms.TypeCast(mindspore.int32)

    # map给定一组数据增强列表，按顺序将数据增强作用在数据集对象上。
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    # 将数据集中连续 batch_size 条数据组合为一个批数据
    dataset = dataset.batch(batch_size)
    return dataset
# 对数据集进行transfrom和batch
train_dataset = datapipe(train_dataset, BATCH_SIZE)
test_dataset = datapipe(test_dataset, BATCH_SIZE)

# 5. 实验过程


## 5.1 导入所需库和函数
在代码最开始集中导入整个实验中所需要使用的库和函数。

In [128]:
# 通用数据增强
from mindspore.dataset import transforms
# 读取和解析Manifest数据文件构建数据集
from mindspore.dataset import MnistDataset

## 5.2 模型构建
使用mindspore中提供的相关函数构建训练使用的神经网络模型，包括平图层，致密连接层等，并在construct中完成神经网络的前向构造。\
为了对比dropout层对深度学习的影响，做如下设置：
1. 构建两个除了dropout层完全相同的网络模型。
2. 采用完全相同的超参数及相关设置。

In [129]:
# 定义附加dropout层的模型
# MindSpore 中提供用户通过继承 nn.Cell 来方便用户创建和执行自己的网络
class Network(nn.Cell): 
    # 自定义的网络中，需要在__init__构造函数中申明各个层的定义
    def __init__(self): 
         # 继承父类nn.cell的__init__方法
        super().__init__()         
        # nn.Flatten为输入展成平图层，即去掉那些空的维度
        self.flatten = nn.Flatten()
        # 使用SequentialCell对网络进行管理
        self.dense_relu_sequential = nn.SequentialCell(
            # nn.Dense为致密连接层，它的第一个参数为输入层的维度，第二个参数为输出的维度，
            # 第三个参数为神经网络可训练参数W权重矩阵的初始化方式，默认为normal
            # nn.ReLU()非线性激活函数，它往往比论文中的sigmoid激活函数具有更好的效益
            nn.Dense(28 * 28, 512), # 致密连接层 输入28*28 输出512
            nn.ReLU(),              # ReLU层
            nn.Dense(512, 512),     # 致密连接层 输入512 输出512
            nn.Dropout(keep_prob=0.5),# Dropout层，舍弃率为0.5
            nn.ReLU(),              # ReLu层
            # nn.Dropout(keep_prob=0.1),# Dropout层，舍弃率为0.1
            nn.Dense(512, 10)       # 致密连接层 输入512 输出10
            # nn.Dropout(keep_prob=0.1)# Dropout层，舍弃率为0.1
        
        )
    # 在construct中实现层之间的连接关系，完成神经网络的前向构造
    def construct(self, x):
         #调用init中定义的self.flatten()方法
        x = self.flatten(x)
        #调用init中的self.dense_relu_sequential()方法
        logits = self.dense_relu_sequential(x)
        # 返回模型
        return logits
model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): Dropout<keep_prob=0.5>
    (4): ReLU<>
    (5): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


In [130]:
# 定义不附加dropout层的模型
class Network2(nn.Cell): 
    # 自定义的网络中，需要在__init__构造函数中申明各个层的定义
    def __init__(self): 
         # 继承父类nn.cell的__init__方法
        super().__init__()         
        # nn.Flatten为输入展成平图层，即去掉那些空的维度
        self.flatten = nn.Flatten()
        # 使用SequentialCell对网络进行管理
        self.dense_relu_sequential = nn.SequentialCell(
            # nn.Dense为致密连接层，它的第一个参数为输入层的维度，第二个参数为输出的维度，
            # 第三个参数为神经网络可训练参数W权重矩阵的初始化方式，默认为normal
            # nn.ReLU()非线性激活函数，它往往比论文中的sigmoid激活函数具有更好的效益
            nn.Dense(28 * 28, 512), # 致密连接层 输入28*28 输出512
            # nn.Dropout(keep_prob=0.3),# Dropout层，舍弃率为0.3
            nn.ReLU(),              # ReLU层
            nn.Dense(512, 512),     # 致密连接层 输入512 输出512
            nn.ReLU(),              # ReLu层
            nn.Dense(512, 10)       # 致密连接层 输入512 输出10
        
        )
    # 在construct中实现层之间的连接关系，完成神经网络的前向构造
    def construct(self, x):
         #调用init中定义的self.flatten()方法
        x = self.flatten(x)
        #调用init中的self.dense_relu_sequential()方法
        logits = self.dense_relu_sequential(x)
        # 返回模型
        return logits
model2 = Network2()
print(model2)

Network2<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


# 6、模型训练

在模型训练中，一个完整的训练过程（step）需要实现以下三步：
1. 正向计算：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。
2. 反向传播：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。
3. 参数优化：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：
1. 正向计算函数定义。
2. 通过函数变换获得梯度计算函数。
3. 训练函数定义，执行正向计算、反向传播和参数优化。

为对比增加dropout层和普通定义模型，进行如下设置：
1. 构建完全相同的梯度下降方法分别训练两个模型。
2. 采用完全相同的数据集以及相同的分割比例。
3. 采用相同的交叉熵损失函数。

In [131]:
# 实例化损失函数和优化器
# 计算预测值和目标值之间的交叉熵损失
loss_fn = nn.CrossEntropyLoss() 
#构建一个Optimizer对象，能够保持当前参数状态并基于计算得到的梯度进行参数更新 此处使用随机梯度下降算法
optimizer = nn.SGD(model.trainable_params(), learning_rate=LEARNING_RATE) 
optimizer2 = nn.SGD(model2.trainable_params(), learning_rate=LEARNING_RATE) 

In [132]:
def train(model, dataset, loss_fn, optimizer):
    # 定义 forward 函数
    def forward_fn(data, label):
        # 将数据载入模型
        logits = model(data)
        # 根据模型训练获取损失函数值
        loss = loss_fn(logits, label)
        return loss, logits
    # 调用梯度函数，value_and_grad()为生成求导函数，用于计算给定函数的正向计算结果和梯度
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
    # 定义一步训练的函数
    def train_step(data, label):
        # 计算梯度，记录变量是怎么来的
        (loss, _), grads = grad_fn(data, label)
        # 获得损失 depend用来处理操作间的依赖关系
        loss = ops.depend(loss, optimizer(grads))
        return loss
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        # 批量训练获得损失值
        loss = train_step(data, label)
        # 当完成所有数据样本的训练
        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In [133]:
def train(model2, dataset, loss_fn, optimizer2):
    # 定义 forward 函数
    def forward_fn(data, label):
        # 将数据载入模型
        logits = model2(data)
        # 根据模型训练获取损失函数值
        loss = loss_fn(logits, label)
        return loss, logits
    # 调用梯度函数，value_and_grad()为生成求导函数，用于计算给定函数的正向计算结果和梯度
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer2.parameters, has_aux=True)
    # 定义一步训练的函数
    def train_step(data, label):
        # 计算梯度，记录变量是怎么来的
        (loss, _), grads = grad_fn(data, label)
        # 获得损失 depend用来处理操作间的依赖关系
        loss = ops.depend(loss, optimizer2(grads))
        return loss
    size = dataset.get_dataset_size()
    model2.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        # 批量训练获得损失值
        loss = train_step(data, label)
        # 当完成所有数据样本的训练
        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

定义完全相同的两个测试函数，通过调用mindspore中的相关函数，使用accuracy以及平均损失等指标评估预测模型的性能。

In [134]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator(): # 遍历所有测试样本数据
        pred = model(data)                              # 根据已训练模型获取预测值
        total += len(data)                              # 统计样本数
        test_loss += loss_fn(pred, label).asnumpy()     # 统计样本损失值
        correct += (pred.argmax(1) == label).asnumpy().sum()# 统计预测正确的样本个数
    test_loss /= num_batches                              # 求得平均损失
    correct /= total                                      # 计算accuracy
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [135]:
def test(model2, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model2.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator(): # 遍历所有测试样本数据
        pred = model2(data)                              # 根据已训练模型获取预测值
        total += len(data)                              # 统计样本数
        test_loss += loss_fn(pred, label).asnumpy()     # 统计样本损失值
        correct += (pred.argmax(1) == label).asnumpy().sum()# 统计预测正确的样本个数
    test_loss /= num_batches                              # 求得平均损失
    correct /= total                                      # 计算accuracy
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程需多次迭代数据集，一次完整的迭代称为一轮（epoch）。在每一轮，遍历训练集进行训练，结束后使用测试集进行预测。打印每一轮的loss值和预测准确率（Accuracy），可以看到loss在不断下降，Accuracy在不断提高。\
相比较没有dropout层的模型，可以看到在测试集中，搭建了dropout层的模型准确率更高，损失函数更小。

In [136]:
for t in range(EPOCH):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)      # 训练模型
    test(model, test_dataset, loss_fn)                   # 测试模型
print("finished!")

Epoch 1
-------------------------------
loss: 2.303168  [  0/938]
loss: 2.290722  [100/938]
loss: 2.277732  [200/938]
loss: 2.217930  [300/938]
loss: 2.032400  [400/938]
loss: 1.600762  [500/938]
loss: 1.086087  [600/938]
loss: 0.924158  [700/938]
loss: 0.985418  [800/938]
loss: 0.657237  [900/938]
Test: 
 Accuracy: 84.8%, Avg loss: 0.551248 

Epoch 2
-------------------------------
loss: 0.485563  [  0/938]
loss: 0.606317  [100/938]
loss: 0.357362  [200/938]
loss: 0.529909  [300/938]
loss: 0.351772  [400/938]
loss: 0.438352  [500/938]
loss: 0.358124  [600/938]
loss: 0.332275  [700/938]
loss: 0.505310  [800/938]
loss: 0.164845  [900/938]
Test: 
 Accuracy: 90.2%, Avg loss: 0.339299 

Epoch 3
-------------------------------
loss: 0.287869  [  0/938]
loss: 0.491125  [100/938]
loss: 0.423553  [200/938]
loss: 0.253829  [300/938]
loss: 0.354354  [400/938]
loss: 0.162713  [500/938]
loss: 0.355667  [600/938]
loss: 0.332413  [700/938]
loss: 0.423598  [800/938]
loss: 0.537284  [900/938]
Test: 
 

In [137]:
for i in range(EPOCH):
    print(f"Epoch {i+1}\n-------------------------------")
    train(model2, train_dataset, loss_fn, optimizer2)      # 训练模型
    test(model2, test_dataset, loss_fn)                   # 测试模型
print("finished!")

Epoch 1
-------------------------------
loss: 2.303347  [  0/938]
loss: 2.286153  [100/938]
loss: 2.254628  [200/938]
loss: 2.194401  [300/938]
loss: 1.928651  [400/938]
loss: 1.374302  [500/938]
loss: 0.980530  [600/938]
loss: 0.810860  [700/938]
loss: 0.499896  [800/938]
loss: 0.525144  [900/938]
Test: 
 Accuracy: 85.1%, Avg loss: 0.521591 

Epoch 2
-------------------------------
loss: 0.639420  [  0/938]
loss: 0.377849  [100/938]
loss: 0.252687  [200/938]
loss: 0.383685  [300/938]
loss: 0.326278  [400/938]
loss: 0.389681  [500/938]
loss: 0.321718  [600/938]
loss: 0.241322  [700/938]
loss: 0.410330  [800/938]
loss: 0.226820  [900/938]
Test: 
 Accuracy: 90.1%, Avg loss: 0.338876 

Epoch 3
-------------------------------
loss: 0.291637  [  0/938]
loss: 0.255920  [100/938]
loss: 0.336062  [200/938]
loss: 0.433291  [300/938]
loss: 0.375580  [400/938]
loss: 0.225151  [500/938]
loss: 0.601096  [600/938]
loss: 0.500888  [700/938]
loss: 0.097533  [800/938]
loss: 0.405210  [900/938]
Test: 
 

# 7、模型预测

模型训练完成后，将训练好的模型保存至指定的路径，方便后续的实验中在使用该模型的时候能够加载模型。\
实例化一个随机初始化的模型，并将训练好的超参数加载到刚才初始化的模型中。\
使用测试数据测试模型的识别能力。

In [138]:
# 保存checkpoint时的配置策略
mindspore.save_checkpoint(model, "model.ckpt")
mindspore.save_checkpoint(model2, "model2.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


In [147]:
# 实例化随机初始化的模型 
model = Network()
model2 = Network2()
# 加载检查点，加载参数到模型 
param_dict = mindspore.load_checkpoint("model.ckpt")
param_dict2 = mindspore.load_checkpoint("model2.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)
param_not_load2 = mindspore.load_param_into_net(model2, param_dict2)

调用加载dropout层的模型进行预测并显示预测结果和真实结果的对比

In [140]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break

Predicted: "[1 3 7 5 4 5 4 7 8 0]", Actual: "[1 3 7 5 4 5 4 7 8 0]"


调用不加载dropout层的模型进行预测并显示预测结果和真实结果的对比

In [148]:
model2.set_train(False)
for data, label in test_dataset:
    pred = model2(data)
    predicted2 = pred.argmax(1)
    print(f'Predicted: "{predicted2[:10]}", Actual: "{label[:10]}"')
    break

Predicted: "[1 0 0 4 7 2 6 9 0 3]", Actual: "[1 0 0 4 7 2 6 9 0 3]"
